In [3]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

spark_conf = SparkConf()\
  .setAppName("YourTest")\
  .setMaster("local[*]")

sc = SparkContext.getOrCreate(spark_conf)

# Load packages
from pyspark.sql import SparkSession
from pyspark import SparkContext

# Connect to the local spark instance using as many threads as on your machine
spark = SparkSession.builder.master("local[*]").appName("Data_processing").getOrCreate()
print(spark.version)

import pyspark.sql.functions as F
from pyspark.sql.types import *

3.2.1


In [5]:
schema = StructType() \
    .add("user_id", 'string') \
    .add("country", "string") \
    .add("browser", "string") \
    .add('OS', "string") \
    .add("age", "integer")

In [7]:
df = spark.createDataFrame([
    ("A203", 'india', 'Chrome', "WIN", 33),
    ("A201", "China", "Safari", "MacOS", 35),
    ("A205", "UK", "Mozilla", "Linux", 25),
], schema=schema)

In [8]:
df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- country: string (nullable = true)
 |-- browser: string (nullable = true)
 |-- OS: string (nullable = true)
 |-- age: integer (nullable = true)



In [9]:
df.show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A203|  india| Chrome|  WIN| 33|
|   A201|  China| Safari|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



# Null Values

In [10]:
#create a new dataframe with null values 
df_na=spark.createDataFrame([
    ("A203",None,"Chrome","WIN",33),
    ("A201",'China',None,"MacOS",35),
    ("A205",'UK',"Mozilla","Linux",25)
],schema=schema)

In [11]:
df_na.show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A203|   null| Chrome|  WIN| 33|
|   A201|  China|   null|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [14]:
df_na.fillna('0').show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A203|      0| Chrome|  WIN| 33|
|   A201|  China|      0|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [15]:
df_na.fillna({'country': 'USA', 'browser': 'Safari'}).show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A203|    USA| Chrome|  WIN| 33|
|   A201|  China| Safari|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [16]:
df_na.na.drop().show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [17]:
df_na.na.drop(subset='country').show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A201|  China|   null|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [18]:
df_na.replace('Chrome', 'Google Chrome').show()

+-------+-------+-------------+-----+---+
|user_id|country|      browser|   OS|age|
+-------+-------+-------------+-----+---+
|   A203|   null|Google Chrome|  WIN| 33|
|   A201|  China|         null|MacOS| 35|
|   A205|     UK|      Mozilla|Linux| 25|
+-------+-------+-------------+-----+---+



In [19]:
df_na.drop('user_id').show()

+-------+-------+-----+---+
|country|browser|   OS|age|
+-------+-------+-----+---+
|   null| Chrome|  WIN| 33|
|  China|   null|MacOS| 35|
|     UK|Mozilla|Linux| 25|
+-------+-------+-----+---+



# Read files as DataFrame

In [20]:
df = spark.read.csv('customer_data.csv', header=True, inferSchema=True)

In [21]:
df.count()

2000

In [22]:
len(df.columns)

7

In [23]:
df.printSchema()

root
 |-- Customer_subtype: string (nullable = true)
 |-- Number_of_houses: integer (nullable = true)
 |-- Avg_size_household: integer (nullable = true)
 |-- Avg_age: string (nullable = true)
 |-- Customer_main_type: string (nullable = true)
 |-- Avg_Salary: integer (nullable = true)
 |-- label: integer (nullable = true)



In [24]:
df.show(3)

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|Lower class large...|               1|                 3|30-40 years|Family with grown...|     44905|    0|
|Mixed small town ...|               1|                 2|30-40 years|Family with grown...|     37575|    0|
|Mixed small town ...|               1|                 2|30-40 years|Family with grown...|     27915|    0|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
only showing top 3 rows



In [25]:
df.summary().show()

+-------+--------------------+------------------+------------------+-----------+--------------------+-----------------+------------------+
|summary|    Customer_subtype|  Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|       Avg_Salary|             label|
+-------+--------------------+------------------+------------------+-----------+--------------------+-----------------+------------------+
|  count|                2000|              2000|              2000|       2000|                2000|             2000|              2000|
|   mean|                null|            1.1075|            2.6895|       null|                null|     1616908.0835|            0.0605|
| stddev|                null|0.3873225521186316|0.7914562220841646|       null|                null|6822647.757312146|0.2384705099001677|
|    min|Affluent senior a...|                 1|                 1|20-30 years|      Average Family|             1361|                 0|
|    25%|                nu

# Subset of a DataFrame

In [27]:
df.select(['Customer_subtype', 'Avg_Salary']).show()

+--------------------+----------+
|    Customer_subtype|Avg_Salary|
+--------------------+----------+
|Lower class large...|     44905|
|Mixed small town ...|     37575|
|Mixed small town ...|     27915|
|Modern, complete ...|     19504|
|  Large family farms|     34943|
|    Young and rising|     13064|
|Large religious f...|     29090|
|Lower class large...|      6895|
|Lower class large...|     35497|
|     Family starters|     30800|
|       Stable family|     39157|
|Modern, complete ...|     40839|
|Lower class large...|     30008|
|        Mixed rurals|     37209|
|    Young and rising|     45361|
|Lower class large...|     45650|
|Traditional families|     18982|
|Mixed apartment d...|     30093|
|Young all america...|     27097|
|Low income catholics|     23511|
+--------------------+----------+
only showing top 20 rows



In [28]:
df.filter((df['Avg_Salary'] > 1000000)).count()

128

In [29]:
df.filter((df['Avg_Salary'] > 1000000)).show()

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
| High status seniors|               1|                 3|40-50 years|Successful hedonists|   4670288|    0|
| High status seniors|               1|                 3|50-60 years|Successful hedonists|   9561873|    0|
| High status seniors|               1|                 2|40-50 years|Successful hedonists|  18687005|    0|
| High status seniors|               1|                 2|40-50 years|Successful hedonists|  24139960|    0|
| High status seniors|               1|                 2|50-60 years|Successful hedonists|   6718606|    0|
|High Income, expe...|               1|                 3|40-50 years|Successful hedonists|  19347139|    0|
|High Income, expe.

# Filter

In [30]:
df.filter(df['Avg_Salary'] > 500000).filter(df['Number_of_houses'] > 2).show()

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    596723|    0|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    944444|    0|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    788477|    0|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    994077|    0|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+



# Where

In [31]:
df.where((df['Avg_Salary'] > 500000) & (df['Number_of_houses'] > 2)).show()

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    596723|    0|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    944444|    0|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    788477|    0|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    994077|    0|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+



# Aggregations

In [32]:
df.groupBy('Customer_subtype').count().show()

+--------------------+-----+
|    Customer_subtype|count|
+--------------------+-----+
|Large family, emp...|   56|
|Religious elderly...|   47|
|Large religious f...|  107|
|Modern, complete ...|   93|
|    Village families|   68|
|Young all america...|   62|
|Young urban have-...|    4|
|Young seniors in ...|   22|
|Fresh masters in ...|    2|
|High Income, expe...|   52|
|Lower class large...|  288|
| Residential elderly|    6|
|Senior cosmopolitans|    1|
|        Mixed rurals|   67|
|Career and childcare|   33|
|Low income catholics|   72|
|Mixed apartment d...|   34|
|Seniors in apartm...|   17|
|Middle class fami...|  122|
|Traditional families|  129|
+--------------------+-----+
only showing top 20 rows



In [33]:
for col in df.columns:
    if col != 'Avg_Salary':
        print(f'Aggregation for {col}')
        df.groupBy(col).count().orderBy('count', ascending=False).show(truncate=False)

Aggregation for Customer_subtype
+------------------------------------------+-----+
|Customer_subtype                          |count|
+------------------------------------------+-----+
|Lower class large families                |288  |
|Traditional families                      |129  |
|Middle class families                     |122  |
|Large religious families                  |107  |
|Modern, complete families                 |93   |
|Couples with teens 'Married with children'|83   |
|Young and rising                          |78   |
|High status seniors                       |76   |
|Low income catholics                      |72   |
|Mixed seniors                             |71   |
|Village families                          |68   |
|Mixed rurals                              |67   |
|Young all american family                 |62   |
|Stable family                             |62   |
|Large family, employed child              |56   |
|Young, low educated                       |56   

In [34]:
df.groupBy('Customer_main_type').agg(F.mean('Avg_Salary')).show()

+--------------------+--------------------+
|  Customer_main_type|     avg(Avg_Salary)|
+--------------------+--------------------+
|             Farmers|  30209.333333333332|
|       Career Loners|             32272.6|
|Retired and Relig...|   27338.80693069307|
|Successful hedonists|1.6278923510309279E7|
|         Living well|  31194.044943820223|
|      Average Family|  104256.62337662338|
|    Cruising Seniors|  28870.333333333332|
|Conservative fami...|  29504.419491525423|
|      Driven Growers|   30769.04069767442|
|Family with grown...|  28114.191881918818|
+--------------------+--------------------+



In [35]:
df.groupBy('Customer_main_type').agg(F.max('Avg_Salary')).show()

+--------------------+---------------+
|  Customer_main_type|max(Avg_Salary)|
+--------------------+---------------+
|             Farmers|          49965|
|       Career Loners|          49903|
|Retired and Relig...|          49564|
|Successful hedonists|       48919896|
|         Living well|          49816|
|      Average Family|         991838|
|    Cruising Seniors|          49526|
|Conservative fami...|          49965|
|      Driven Growers|          49932|
|Family with grown...|          49901|
+--------------------+---------------+



In [36]:
df.groupBy('Customer_main_type').agg(F.min('Avg_Salary')).show()

+--------------------+---------------+
|  Customer_main_type|min(Avg_Salary)|
+--------------------+---------------+
|             Farmers|          10469|
|       Career Loners|          13246|
|Retired and Relig...|           1361|
|Successful hedonists|          12705|
|         Living well|          10418|
|      Average Family|          10506|
|    Cruising Seniors|          10100|
|Conservative fami...|          10179|
|      Driven Growers|          10257|
|Family with grown...|           1502|
+--------------------+---------------+



In [37]:
df.groupBy('Customer_main_type').agg(F.sum('Avg_Salary')).show()

+--------------------+---------------+
|  Customer_main_type|sum(Avg_Salary)|
+--------------------+---------------+
|             Farmers|        2809468|
|       Career Loners|         484089|
|Retired and Relig...|        5522439|
|Successful hedonists|     3158111161|
|         Living well|        5552540|
|      Average Family|       32111040|
|    Cruising Seniors|        1732220|
|Conservative fami...|        6963043|
|      Driven Growers|        5292275|
|Family with grown...|       15237892|
+--------------------+---------------+



In [38]:
df.sort("Avg_Salary", ascending=False).show()

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+
| High status seniors|               1|                 2|60-70 years|Successful hedonists|  48919896|    0|
|High Income, expe...|               1|                 2|50-60 years|Successful hedonists|  48177970|    0|
|High Income, expe...|               1|                 2|50-60 years|Successful hedonists|  48069548|    1|
|High Income, expe...|               1|                 3|40-50 years|Successful hedonists|  46911924|    0|
| High status seniors|               1|                 3|40-50 years|Successful hedonists|  46614009|    0|
|High Income, expe...|               1|                 3|30-40 years|Successful hedonists|  45952441|    0|
|High Income, expe.

In [39]:
df.groupBy('Customer_subtype') \
    .agg(F.avg('Avg_Salary').alias('mean_salary')) \
    .orderBy('mean_salary', ascending=False) \
    .show(50, False)

+------------------------------------------+--------------------+
|Customer_subtype                          |mean_salary         |
+------------------------------------------+--------------------+
|High status seniors                       |2.507677857894737E7 |
|High Income, expensive child              |2.3839817807692308E7|
|Affluent young families                   |662068.7777777778   |
|Affluent senior apartments                |653638.8235294118   |
|Senior cosmopolitans                      |49903.0             |
|Students in apartments                    |35532.142857142855  |
|Large family farms                        |33135.61538461538   |
|Young, low educated                       |33072.21428571428   |
|Large family, employed child              |32867.857142857145  |
|Suburban youth                            |32558.0             |
|Village families                          |32449.470588235294  |
|Middle class families                     |31579.385245901638  |
|Modern, c

In [40]:
df.groupBy('Customer_subtype') \
    .agg(F.max('Avg_Salary').alias('max_salary')) \
    .orderBy('max_salary', ascending=False) \
    .show()

+--------------------+----------+
|    Customer_subtype|max_salary|
+--------------------+----------+
| High status seniors|  48919896|
|High Income, expe...|  48177970|
|Affluent senior a...|    994077|
|Affluent young fa...|    991838|
|Traditional families|     49965|
|  Large family farms|     49965|
|Middle class fami...|     49932|
|Senior cosmopolitans|     49903|
|Mixed small town ...|     49901|
|Lower class large...|     49899|
|       Mixed seniors|     49876|
|    Young and rising|     49816|
|        Mixed rurals|     49785|
|Modern, complete ...|     49729|
| Young, low educated|     49626|
|Mixed apartment d...|     49621|
|     Family starters|     49602|
|    Village families|     49575|
|Religious elderly...|     49564|
|       Stable family|     49548|
+--------------------+----------+
only showing top 20 rows



# Collect

In [42]:
df.groupBy('Customer_subtype').agg(F.collect_set('Number_of_houses')).show()

+--------------------+-----------------------------+
|    Customer_subtype|collect_set(Number_of_houses)|
+--------------------+-----------------------------+
|Large family, emp...|                       [1, 2]|
|Religious elderly...|                       [1, 2]|
|Large religious f...|                       [1, 2]|
|Modern, complete ...|                       [1, 2]|
|    Village families|                       [1, 2]|
|Young all america...|                       [1, 2]|
|Young urban have-...|                       [1, 2]|
|Young seniors in ...|                    [1, 2, 3]|
|Fresh masters in ...|                          [1]|
|High Income, expe...|                          [1]|
|Lower class large...|                       [1, 2]|
| Residential elderly|                    [1, 2, 3]|
|Senior cosmopolitans|                          [3]|
|        Mixed rurals|                          [1]|
|Career and childcare|                       [1, 2]|
|Low income catholics|                        

In [43]:
df.groupBy('Customer_subtype').agg(F.collect_list('Number_of_houses')).show()

+--------------------+------------------------------+
|    Customer_subtype|collect_list(Number_of_houses)|
+--------------------+------------------------------+
|Large family, emp...|          [2, 1, 2, 1, 2, 1...|
|Religious elderly...|          [1, 1, 1, 1, 1, 1...|
|Large religious f...|          [2, 1, 1, 2, 1, 1...|
|Modern, complete ...|          [1, 1, 2, 1, 1, 1...|
|    Village families|          [1, 1, 1, 1, 1, 1...|
|Young all america...|          [1, 1, 2, 2, 1, 1...|
|Young urban have-...|                  [1, 2, 1, 1]|
|Young seniors in ...|          [1, 1, 1, 1, 1, 2...|
|Fresh masters in ...|                        [1, 1]|
|High Income, expe...|          [1, 1, 1, 1, 1, 1...|
|Lower class large...|          [1, 1, 1, 1, 1, 1...|
| Residential elderly|            [3, 1, 1, 3, 2, 1]|
|Senior cosmopolitans|                           [3]|
|        Mixed rurals|          [1, 1, 1, 1, 1, 1...|
|Career and childcare|          [2, 1, 1, 1, 1, 1...|
|Low income catholics|      

In [44]:
df = df.withColumn('constant', F.lit('finance'))
df.select('Customer_subtype', 'constant').show()

+--------------------+--------+
|    Customer_subtype|constant|
+--------------------+--------+
|Lower class large...| finance|
|Mixed small town ...| finance|
|Mixed small town ...| finance|
|Modern, complete ...| finance|
|  Large family farms| finance|
|    Young and rising| finance|
|Large religious f...| finance|
|Lower class large...| finance|
|Lower class large...| finance|
|     Family starters| finance|
|       Stable family| finance|
|Modern, complete ...| finance|
|Lower class large...| finance|
|        Mixed rurals| finance|
|    Young and rising| finance|
|Lower class large...| finance|
|Traditional families| finance|
|Mixed apartment d...| finance|
|Young all america...| finance|
|Low income catholics| finance|
+--------------------+--------+
only showing top 20 rows



# User-Defined Functions

In [ ]:
from pyspark.sql.functions import udf

In [45]:
df.groupBy('Avg_age').count().show()

+-----------+-----+
|    Avg_age|count|
+-----------+-----+
|70-80 years|    8|
|50-60 years|  373|
|30-40 years|  496|
|20-30 years|   31|
|60-70 years|   64|
|40-50 years| 1028|
+-----------+-----+



In [46]:
def age_category(age: str) -> str:
    if age=='20-30 years':
        return 'Young'
    elif age=="30-40 years":
        return 'Mid Aged'
    elif ((age=="40-50 years") or (age=="50-60 years")):
        return 'Old'
    else:
        return 'Very Old'

In [47]:
age_udf = udf(age_category, StringType())

In [48]:
df = df.withColumn('age_category', age_udf(df['Avg_age']))
df.select('Avg_age', 'age_category').show()

+-----------+------------+
|    Avg_age|age_category|
+-----------+------------+
|30-40 years|    Mid Aged|
|30-40 years|    Mid Aged|
|30-40 years|    Mid Aged|
|40-50 years|         Old|
|30-40 years|    Mid Aged|
|20-30 years|       Young|
|30-40 years|    Mid Aged|
|40-50 years|         Old|
|50-60 years|         Old|
|40-50 years|         Old|
|40-50 years|         Old|
|40-50 years|         Old|
|40-50 years|         Old|
|40-50 years|         Old|
|30-40 years|    Mid Aged|
|40-50 years|         Old|
|40-50 years|         Old|
|40-50 years|         Old|
|30-40 years|    Mid Aged|
|50-60 years|         Old|
+-----------+------------+
only showing top 20 rows



In [49]:
df.groupBy('age_category').count().show()

+------------+-----+
|age_category|count|
+------------+-----+
|    Mid Aged|  496|
|    Very Old|   72|
|         Old| 1401|
|       Young|   31|
+------------+-----+



# Pandas UDF

In [50]:
df.select('Avg_Salary').summary().show()

+-------+-----------------+
|summary|       Avg_Salary|
+-------+-----------------+
|  count|             2000|
|   mean|     1616908.0835|
| stddev|6822647.757312146|
|    min|             1361|
|    25%|            20315|
|    50%|            31421|
|    75%|            42949|
|    max|         48919896|
+-------+-----------------+



In [80]:
max_sal, min_sal = df.select('Avg_Salary').agg(F.max('Avg_Salary'), F.min('Avg_Salary')).collect()[0]
# max_sal, min_sal = 

In [81]:
max_sal, min_sal

(48919896, 1361)

In [94]:
type(max_sal)

int

In [97]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.column import Column

In [92]:
def scaled_salary(salary: Column, min_sal: Column, max_sal: Column):
    return (salary-min_sal)/(max_sal-min_sal)

In [95]:
scaling_udf = pandas_udf(scaled_salary, DoubleType())
df.withColumn("scaled_salary", scaling_udf(df['Avg_Salary'], F.lit(min_sal), F.lit(max_sal))).show(10, False)

+--------------------------+----------------+------------------+-----------+---------------------+----------+-----+--------+------------+---------------------+
|Customer_subtype          |Number_of_houses|Avg_size_household|Avg_age    |Customer_main_type   |Avg_Salary|label|constant|age_category|scaled_salary        |
+--------------------------+----------------+------------------+-----------+---------------------+----------+-----+--------+------------+---------------------+
|Lower class large families|1               |3                 |30-40 years|Family with grown ups|44905     |0    |finance |Mid Aged    |8.901329526732557E-4 |
|Mixed small town dwellers |1               |2                 |30-40 years|Family with grown ups|37575     |0    |finance |Mid Aged    |7.40291997705982E-4  |
|Mixed small town dwellers |1               |2                 |30-40 years|Family with grown ups|27915     |0    |finance |Mid Aged    |5.42820834679534E-4  |
|Modern, complete families |1           

# Joins

In [100]:
region_data = spark.createDataFrame([
    ('Family with grown ups','PN'),
    ('Driven Growers','GJ'),
    ('Conservative families','DD'),
    ('Cruising Seniors','DL'),
    ('Average Family ','MN'),
    ('Living well','KA'),
    ('Successful hedonists','JH'),
    ('Retired and Religious','AX'),
    ('Career Loners','HY'),('Farmers','JH'),
], schema=StructType().add("Customer_main_type", "string").add("Region Code", "string"))

In [101]:
region_data.show()

+--------------------+-----------+
|  Customer_main_type|Region Code|
+--------------------+-----------+
|Family with grown...|         PN|
|      Driven Growers|         GJ|
|Conservative fami...|         DD|
|    Cruising Seniors|         DL|
|     Average Family |         MN|
|         Living well|         KA|
|Successful hedonists|         JH|
|Retired and Relig...|         AX|
|       Career Loners|         HY|
|             Farmers|         JH|
+--------------------+-----------+



In [102]:
new_df = df.join(region_data, on="Customer_main_type")

In [103]:
new_df.groupBy('Region Code').count().show()

+-----------+-----+
|Region Code|count|
+-----------+-----+
|         PN|  542|
|         GJ|  172|
|         DD|  236|
|         DL|   60|
|         KA|  178|
|         JH|  287|
|         AX|  202|
|         HY|   15|
+-----------+-----+



# Pivoting

In [104]:
df.groupBy('Customer_main_type') \
    .pivot('Avg_age') \
    .sum('Avg_Salary') \
    .fillna(0) \
    .show()

+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+
|  Customer_main_type|20-30 years|30-40 years|40-50 years|50-60 years|60-70 years|70-80 years|
+--------------------+-----------+-----------+-----------+-----------+-----------+-----------+
|             Farmers|          0|     462027|    2031235|     316206|          0|          0|
|       Career Loners|     143998|     176639|      25701|     105193|      32558|          0|
|Retired and Relig...|     126350|     336631|    2975266|    1687711|     335357|      61124|
|Successful hedonists|      42261|  171278764| 1223362814| 1563071675|  200340129|      15518|
|         Living well|     460528|    2965303|    1795405|     331304|          0|          0|
|      Average Family|          0|   23682805|    7789464|     412490|     226281|          0|
|    Cruising Seniors|          0|      43302|     303601|     529354|     716425|     139538|
|Conservative fami...|      69390|    2381485|    

In [105]:
df.groupBy('Customer_main_type') \
    .pivot('label') \
    .sum('Avg_Salary') \
    .fillna(0) \
    .show()

+--------------------+----------+---------+
|  Customer_main_type|         0|        1|
+--------------------+----------+---------+
|             Farmers|   2734832|    74636|
|       Career Loners|    484089|        0|
|Retired and Relig...|   5328410|   194029|
|Successful hedonists|2720381462|437729699|
|         Living well|   5453384|    99156|
|      Average Family|  26036999|  6074041|
|    Cruising Seniors|   1675841|    56379|
|Conservative fami...|   6595027|   368016|
|      Driven Growers|   4492465|   799810|
|Family with grown...|  14394094|   843798|
+--------------------+----------+---------+



# Window Functions or Windowed Aggregates

In [106]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

In [107]:
win = Window.orderBy(df['Avg_Salary'].desc())

In [108]:
df = df.withColumn('rank', row_number().over(win)).alias('rank')

In [109]:
df.show()

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+------------+----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|constant|age_category|rank|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+------------+----+
| High status seniors|               1|                 2|60-70 years|Successful hedonists|  48919896|    0| finance|    Very Old|   1|
|High Income, expe...|               1|                 2|50-60 years|Successful hedonists|  48177970|    0| finance|         Old|   2|
|High Income, expe...|               1|                 2|50-60 years|Successful hedonists|  48069548|    1| finance|         Old|   3|
|High Income, expe...|               1|                 3|40-50 years|Successful hedonists|  46911924|    0| finance|         Old|   4|
| High status seniors|               1|         

In [110]:
win_1 = Window.partitionBy("Customer_subtype").orderBy(df['Avg_Salary'].desc())
df = df.withColumn('rank', row_number().over(win_1).alias('rank'))

In [111]:
df.groupBy('rank').count().orderBy('rank').show()

+----+-----+
|rank|count|
+----+-----+
|   1|   39|
|   2|   37|
|   3|   36|
|   4|   36|
|   5|   34|
|   6|   34|
|   7|   32|
|   8|   31|
|   9|   31|
|  10|   31|
|  11|   31|
|  12|   31|
|  13|   31|
|  14|   31|
|  15|   31|
|  16|   30|
|  17|   30|
|  18|   27|
|  19|   27|
|  20|   27|
+----+-----+
only showing top 20 rows



In [114]:
df.filter(col('rank') < 4).show()

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+------------+----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|constant|age_category|rank|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+------------+----+
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    994077|    0| finance|         Old|   1|
|Affluent senior a...|               1|                 2|50-60 years|Successful hedonists|    983051|    0| finance|         Old|   2|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    944444|    0| finance|         Old|   3|
|Affluent young fa...|               1|                 3|30-40 years|      Average Family|    991838|    0| finance|    Mid Aged|   1|
|Affluent young fa...|               1|         

In [113]:
df.filter(df['rank'] < 4).show()

+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+------------+----+
|    Customer_subtype|Number_of_houses|Avg_size_household|    Avg_age|  Customer_main_type|Avg_Salary|label|constant|age_category|rank|
+--------------------+----------------+------------------+-----------+--------------------+----------+-----+--------+------------+----+
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    994077|    0| finance|         Old|   1|
|Affluent senior a...|               1|                 2|50-60 years|Successful hedonists|    983051|    0| finance|         Old|   2|
|Affluent senior a...|               3|                 2|50-60 years|Successful hedonists|    944444|    0| finance|         Old|   3|
|Affluent young fa...|               1|                 3|30-40 years|      Average Family|    991838|    0| finance|    Mid Aged|   1|
|Affluent young fa...|               1|         